# chatGPT 
* https://openai.com/
* 코드 질문하면 코드 짜주는 웹사이트
* learn about GPT-4
* try on ChatGPT Plus

# 행안부 
## 주소기반산업지원서비스
* https://business.juso.go.kr/addrlink/openApi/apiReqst.do
* API 신청하기
* 도로명주소 공개
* 검색API
* 민간기관
* 업체 - 더조은아카데미신촌
* 시스템명 - 더조은주소개발
* URL - https://ic.tjoeun.co.kr
* 인터넷망, 개발(본인인증없이 발급) 90일
* 신청하기
* 승인키
* 개발 ( 사용기간 : 2023-03-22 ~ 2023-06-20 )

* 메뉴 API신청의 검색API
* 가이드 및 소스 다운로드해서 확인 후 진행하기

In [18]:
# 기본 코드
import pandas as pd
import numpy as np
import requests
from urllib.parse import quote
import json
import folium
import re

In [19]:
# # 코랩 코드 (먼저 실행 후 아래 코드 실행)
# from google.colab import files
# uploaded = files.upload()
# filename = list(uploaded.keys())[0]
# filename # 파일 선택하라고 뜨고 파일을 올려줌

In [20]:
filename = './APIkey/행안부API.txt'
with open(filename) as f:
    api_key =f.read()

In [21]:
len(api_key)

43

In [22]:
# 인터넷으로 무언가를 보내거나 가져올때 쓰는 패키지
import requests
from urllib.parse import quote

In [23]:
# 네이버에서 서울특별시청을 검색해서 나온 주소 복붙
# 쿼리다음에 서울특별시청이 뜨지만 복붙해서 가져오니 quoto 형태로 변환됨
# https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%B2%AD
bldg = '서울특별시청'
quote(bldg)

'%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%B2%AD'

In [24]:
# 예시 (10p - 샘플 데이터 보기 (json))
# https://business.juso.go.kr/addrlink/addrLinkApi.do?currentPage=1&countPerPage=10&keyword=강남대로12길&confmKey=TESTJUSOGOKR&resultType=json
# confmKey=TESTJUSOGOKR&resultType=json

base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'currentPage=1&countPerPage=10&keyword={quote("서울특별시청")}'
params2 = f'confmKey={api_key}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

In [25]:
# # 두번째 코드
# base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
# params1 = f'currentPage=1&countPerPage=10&confmKey={api_key}'
# params2 = f'keyword={quote("서울특별시청")}&resultType=json'
# url = f'{base_url}?{params1}&{params2}'

In [26]:
result = requests.get(url)
result.status_code

200

In [27]:
result.text

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"1","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"본관동,신관","engAddr":"110 Sejong-daero, Jung-gu, Seoul","rn":"세종대로","emdNm":"태평로1가","zipNo":"04524","roadAddrPart2":"(태평로1가)","emdNo":"02","sggNm":"중구","jibunAddr":"서울특별시 중구 태평로1가 31 서울특별시청","siNm":"서울특별시","roadAddrPart1":"서울특별시 중구 세종대로 110","bdNm":"서울특별시청","admCd":"1114010300","udrtYn":"0","lnbrMnnm":"31","roadAddr":"서울특별시 중구 세종대로 110(태평로1가)","lnbrSlno":"0","buldMnnm":"110","bdKdcd":"0","liNm":"","rnMgtSn":"111402005001","mtYn":"0","bdMgtSn":"1114010300100310000019224","buldSlno":"0"}]}}'

In [28]:
import json
res = json.loads(result.text)
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '1',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '본관동,신관',
    'engAddr': '110 Sejong-daero, Jung-gu, Seoul',
    'rn': '세종대로',
    'emdNm': '태평로1가',
    'zipNo': '04524',
    'roadAddrPart2': '(태평로1가)',
    'emdNo': '02',
    'sggNm': '중구',
    'jibunAddr': '서울특별시 중구 태평로1가 31 서울특별시청',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 중구 세종대로 110',
    'bdNm': '서울특별시청',
    'admCd': '1114010300',
    'udrtYn': '0',
    'lnbrMnnm': '31',
    'roadAddr': '서울특별시 중구 세종대로 110(태평로1가)',
    'lnbrSlno': '0',
    'buldMnnm': '110',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '111402005001',
    'mtYn': '0',
    'bdMgtSn': '1114010300100310000019224',
    'buldSlno': '0'}]}}

In [29]:
road_add = res['results']['juso'][0]['roadAddr']
road_add

'서울특별시 중구 세종대로 110(태평로1가)'

In [30]:
places = "종로구청 노원구청 송파구청 마포구청 양천구청"
places = places.split(' ')
places

['종로구청', '노원구청', '송파구청', '마포구청', '양천구청']

In [31]:
# 첫번째 코드
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
params2 = f'confmKey={api_key}&resultType=json'
addr_list = []

for i in places:
    params1 = f'currentPage=1&countPerPage=10&keyword={quote(i)}'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    res = json.loads(result.text)
    road_add = res['results']['juso'][0]['roadAddr']
    addr_list.append(road_add)

In [32]:
# # 두번째 코드
# base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
# params1 = f'currentPage=1&countPerPage=10&confmKey={api_key}'
# addr_list = []

# for i in places:
#     params2 = f'keyword={quote(i)}&resultType=json'
#     url = f'{base_url}?{params1}&{params2}'
#     result = requests.get(url)
#     res = json.loads(result.text)
#     road_add = res['results']['juso'][0]['roadAddr']
#     addr_list.append(road_add)

In [33]:
addr_list

['서울특별시 종로구 종로1길 36(수송동)',
 '서울특별시 노원구 노해로 437(상계동)',
 '서울특별시 송파구 올림픽로 326(신천동)',
 '서울특별시 마포구 월드컵로 지하190(성산동)',
 '서울특별시 양천구 목동동로 105(신정동)']

In [34]:
import pandas as pd
#데이터 프레임으로 저장
df = pd.DataFrame({'공공기관':places, '도로명주소':addr_list})
df.to_csv('공공기관_행안부.csv')

### 강의 외 ###

In [35]:
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'currentPage=1&countPerPage=10&keyword={quote("서울특별시청")}'
params2 = f'&confmKey={api_key}&resultType=json'

url1 = f'{base_url}?{params1}?{params2}'
url2 = f'{base_url}?{params1}&{params2}'

result1 = requests.get(url1)
result1.status_code

result2 = requests.get(url2)
result2.status_code

print(result1.text)

print(result2.text)

{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"1","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"본관동,신관","engAddr":"110 Sejong-daero, Jung-gu, Seoul","rn":"세종대로","emdNm":"태평로1가","zipNo":"04524","roadAddrPart2":"(태평로1가)","emdNo":"02","sggNm":"중구","jibunAddr":"서울특별시 중구 태평로1가 31 서울특별시청","siNm":"서울특별시","roadAddrPart1":"서울특별시 중구 세종대로 110","bdNm":"서울특별시청","admCd":"1114010300","udrtYn":"0","lnbrMnnm":"31","roadAddr":"서울특별시 중구 세종대로 110(태평로1가)","lnbrSlno":"0","buldMnnm":"110","bdKdcd":"0","liNm":"","rnMgtSn":"111402005001","mtYn":"0","bdMgtSn":"1114010300100310000019224","buldSlno":"0"}]}}
{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"1","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"본관동,신관","engAddr":"110 Sejong-daero, Jung-gu, Seoul","rn":"세종대로","emdNm":"태평로1가","zipNo":"04524","roadAddrPart2":"(태평로1가)","emdNo":"02","sggNm":"중구","jibunAddr":"서울특별시 중구 태평로1가 31 서울특별시청","siNm":"서울특별시","roadAddrPart1":"서울특별시 중구 세종대로

In [36]:
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
params2 = f'confmKey={api_key}&resultType=json'
addr_list1 = []

for i in places:
    params1 = f'currentPage=1&countPerPage=10&keyword={quote(i)}'
    url = f'{base_url}?{params1}&{params2}' #f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    res = json.loads(result.text)
    road_add = res['results']['juso'][0]['roadAddr']
    addr_list1.append(road_add)

In [37]:
addr_list1

['서울특별시 종로구 종로1길 36(수송동)',
 '서울특별시 노원구 노해로 437(상계동)',
 '서울특별시 송파구 올림픽로 326(신천동)',
 '서울특별시 마포구 월드컵로 지하190(성산동)',
 '서울특별시 양천구 목동동로 105(신정동)']

# 카카오API
## kakao developers
* 앱이름 / 사용자명
* https://developers.kakao.com/console/app
* 앱 키
* REST API 키
* 메인화면에서 지도/로컬 문서보기
* 유용한 참고 정보
* REST API 레퍼런스
* 로컬 (ctrl+f로 찾기) - 기능 주소 검색하기
* sample

In [48]:
filename = './APIkey/카카오API.txt'
with open(filename) as f:
    api_key = f.read()

In [49]:
len(api_key)

32

In [50]:
import requests
from urllib.parse import quote

In [51]:
addr = '서울특별시 종로구 종로1길 36(수송동)'
quote(addr)

'%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%A2%85%EB%A1%9C1%EA%B8%B8%2036%28%EC%88%98%EC%86%A1%EB%8F%99%29'

In [52]:
# sampel의 코드
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(addr)}'

# "https://dapi.kakao.com/v2/local/search/address.json" \
#   -H "Authorization: KakaoAK ${REST_API_KEY}" \
#   --data-urlencode "query=전북 삼성동 100" 

In [53]:
# json()으로 받는 경우
result_j = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
result_j

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [54]:
# json()으로 받지 않는 경우
result_t = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'})
result_t

<Response [200]>

In [55]:
result_t.status_code

200

In [56]:
result_t.text

'{"documents":[{"address":{"address_name":"서울 종로구 수송동 146-12","b_code":"1111012400","h_code":"1111061500","main_address_no":"146","mountain_yn":"N","region_1depth_name":"서울","region_2depth_name":"종로구","region_3depth_h_name":"종로1.2.3.4가동","region_3depth_name":"수송동","sub_address_no":"12","x":"126.978988255925","y":"37.5735051436739"},"address_name":"서울 종로구 종로1길 36","address_type":"ROAD_ADDR","road_address":{"address_name":"서울 종로구 종로1길 36","building_name":"종로구청","main_building_no":"36","region_1depth_name":"서울","region_2depth_name":"종로구","region_3depth_name":"수송동","road_name":"종로1길","sub_building_no":"","underground_yn":"N","x":"126.978988255925","y":"37.5735051436739","zone_no":"03152"},"x":"126.978988255925","y":"37.5735051436739"}],"meta":{"is_end":true,"pageable_count":1,"total_count":1}}'

In [57]:
# 텍스트에서 json으로 변환
import json
res = json.loads(result_t.text)
res

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [58]:
res.keys()

dict_keys(['documents', 'meta'])

In [59]:
print(res['documents'][0]['x'])
print(res['documents'][0]['y'])

126.978988255925
37.5735051436739


In [60]:
lng = float(res['documents'][0]['x'])
lat = float(res['documents'][0]['y'])
lng, lat

(126.978988255925, 37.5735051436739)

In [61]:
df = pd.read_csv("./data1/공공기관.csv")
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [62]:
lat_list = []
lng_list = []

for i in df.index :
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url, 
                          headers={"Authorization": f'KakaoAK {api_key}'}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

In [63]:
df['위도'] = lat_list
df['경도'] = lng_list
df.to_csv('공공기관2.csv', index=False) # 인덱스를 생성하지 않음

In [64]:
df

,공공기관,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654523,127.056268
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543


In [65]:
import folium

In [66]:
map = folium.Map(location = [df['위도'].mean(), df['경도'].mean()], zoom_start=12)

for i in df.index :
    folium.Circle(radius=300,
                  location = [df['위도'][i], df['경도'][i]],
                  popup = folium.Popup(df['도로명주소'][i], max_width=200),
                  tooltip = df['공공기관'][i],
                  color = 'red',
                  fill = True,
                  fill_color = '#3186cc').add_to(map)
title = '<h3 align="center" style="font-size:20"> 서울시 구청 </h3>'
map.get_root().html.add_child(folium.Element(title))
map